In [66]:
import os
import warnings
import importlib
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, confusion_matrix, f1_score, accuracy_score, \
    precision_score, recall_score, average_precision_score
import xgboost as xgb
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, confusion_matrix, f1_score
from sklearn.model_selection import KFold, LeavePGroupsOut
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from matplotlib import pyplot as plt
import matplotlib
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from tabpfn import TabPFNClassifier
from sklearn.preprocessing import label_binarize
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.utils.class_weight import compute_sample_weight
import shap
# from catboost import CatBoostClassifier

%matplotlib inline
# matplotlib.use('TkAgg')
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler  # install via: pip install imbalanced-learn

warnings.filterwarnings('ignore')
from scipy.stats import skew
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

seed = 69
torch.manual_seed(seed)
np.random.seed(seed)




label = 'agitation-four'       # quaternary     prediction
weighted = True                # weighted      unweighted
kfold = False                   # kfold         lopo

results_dir = '/home/ali/PycharmProjects/tihm/results/06h-quaternary-prediction-lopo-weighted-ctx-sta-phy'

os.makedirs(results_dir, exist_ok=False)

True
NVIDIA GeForce RTX 3080 Ti Laptop GPU


In [67]:
root = '/home/ali/PycharmProjects/tihm/dataset'

dataset = pd.read_csv(os.path.join(root, 'data-06h.csv'))
# dataset_12h = pd.read_csv(os.path.join(root, 'data-12h.csv'))
# dataset_24h = pd.read_csv(os.path.join(root, 'data-24h.csv'))

In [68]:
# display(dataset)

print(dataset['agitation'].equals(dataset['agitation-next']))

print(np.sum(dataset['agitation']))
print(np.sum(dataset['agitation-next']))

print(np.where(dataset['agitation'] == 1))
print(np.where(dataset['agitation-next'] == 1))

print(np.sum(dataset['agitation'] == 0),
      np.sum(dataset['agitation'] == 1))

print(np.sum(dataset['agitation-next'] == 0),
      np.sum(dataset['agitation-next'] == 1))

print(np.sum(dataset['agitation-four'] == 0),
      np.sum(dataset['agitation-four'] == 1),
      np.sum(dataset['agitation-four'] == 2),
      np.sum(dataset['agitation-four'] == 3))


# display(dataset)
print(dataset.columns.values)

False
135.0
135.0
(array([  117,   358,   528,   532,   536,   537,   552,   564,   568,
         572,   597,   661,   669,   923,   935,   940,   944,   963,
         967,   983,  1039,  1044,  1055,  1074,  1662,  2660,  2986,
        3010,  3163,  4157,  4261,  4439,  4442,  4454,  4460,  4464,
        4467,  4468,  4470,  4471,  4472,  4474,  4476,  4480,  4482,
        4483,  4484,  4487,  4488,  4491,  4492,  4495,  4504,  4506,
        4507,  4515,  4519,  4520,  4530,  4532,  4555,  4571,  4579,
        4593,  4692,  4740,  4741,  4748,  4764,  4772,  4792,  4891,
        5066,  5067,  5195,  5386,  5503,  5504,  5523,  5543,  5564,
        5567,  5587,  5591,  5600,  5622,  5626,  5649,  5827,  5828,
        6034,  6120,  6152,  6156,  6164,  6185,  6204,  7192,  7197,
        7241,  7384,  7493,  7501,  7502,  7509,  7517,  7518,  7525,
        7533,  7534,  7537,  7541,  7568,  8661,  8976,  8977,  9070,
        9074,  9087,  9094,  9103,  9186,  9426,  9470,  9514,  9526,
 

In [69]:
# Agitation Statistics

temp = dataset.copy()
# temp = dataset_12h

temp['agitation'].fillna(0, inplace=True)


print(f"Total number of participants: {temp['id'].nunique()}")
print(f"Total number of participants with at least one agitation episode: {temp[temp['agitation'] == 1]['id'].nunique()}")
print(f"Total number of participants with no agitation episodes: {temp['id'].nunique() - temp[temp['agitation'] == 1]['id'].nunique()}")
print(f"Total number of agitation episodes: {(temp['agitation'] == 1).sum()}")


ids, total_counts = np.unique(temp['id'], return_counts=True)
agitated_ids, agitation_counts = np.unique(temp[temp['agitation'] == 1]['id'], return_counts=True)
agitation_map = dict(zip(agitated_ids, agitation_counts))
unique_dates = temp.groupby('id')['date'].nunique().reindex(ids, fill_value=0).values
participant_summary_df = pd.DataFrame({
    'participant_id': ids,
    'six_hour_sample_count': total_counts,
    'unique_dates': unique_dates,
    'agitation_episodes': [agitation_map.get(pid, 0) for pid in ids]
})
participant_summary_df = participant_summary_df.sort_values(by='agitation_episodes', ascending=False).reset_index(drop=True)
# display(participant_summary_df)


values, counts = np.unique(temp.loc[temp['agitation'] == 1, '6h'], return_counts=True)
# values, counts = np.unique(temp.loc[temp['agitation'] == 1, '12h'], return_counts=True)

agitation_temporal_df = pd.DataFrame({
    '6h_time_block': values,
    'agitation_episode_count': counts
})
# display(agitation_temporal_df)

Total number of participants: 56
Total number of participants with at least one agitation episode: 27
Total number of participants with no agitation episodes: 29
Total number of agitation episodes: 135


In [70]:
# 03
columns_indices = ['id', 'date', '6h']

# 08
columns_count = ['back-door', 'bathroom', 'bedroom', 'fridge-door',
                 'front-door', 'hallway', 'kitchen', 'lounge']

# 08
columns_contextual = ['total-events', 'unique-locations', 'active-location-ratio', 'private-to-public-ratio',
                      'location-entropy', 'location-dominance-ratio', 'back-and-forth-count', 'num-transitions']

# 32
columns_statistical = ['back-door-count-max', 'back-door-count-mean', 'back-door-count-std', 'back-door-count-sum',
                       'bathroom-count-max', 'bathroom-count-mean', 'bathroom-count-std', 'bathroom-count-sum',
                       'bedroom-count-max', 'bedroom-count-mean', 'bedroom-count-std', 'bedroom-count-sum',
                       'fridge-door-count-max', 'fridge-door-count-mean', 'fridge-door-count-std', 'fridge-door-count-sum',
                       'front-door-count-max', 'front-door-count-mean', 'front-door-count-std', 'front-door-count-sum',
                       'hallway-count-max', 'hallway-count-mean', 'hallway-count-std', 'hallway-count-sum',
                       'kitchen-count-max', 'kitchen-count-mean', 'kitchen-count-std','kitchen-count-sum',
                       'lounge-count-max', 'lounge-count-mean', 'lounge-count-std','lounge-count-sum']

# 08
columns_physiology = ['body-temperature', 'body-weight', 'diastolic-blood-pressure', 'heart-rate',
                      'muscle-mass', 'systolic-blood-pressure', 'total-body-water', 'skin-temperature']

# 06
columns_labels = ['blood-pressure', 'agitation', 'body-water', 'pulse', 'weight', 'body-temperature-label']

# 02
columns_demographics = ['age', 'sex']

# 02
columns_prediction = ['agitation-next', 'agitation-four']

print(len(columns_count),
      len(columns_statistical),
      len(columns_contextual),
      len(columns_physiology),
      len(columns_demographics),
      len(columns_labels))



8 32 8 8 2 6


In [71]:
columns = columns_indices + columns_count + columns_contextual + columns_statistical + columns_physiology + columns_labels + columns_demographics + columns_prediction
print(dataset.columns.to_list())
print(columns)

print(columns == dataset.columns.to_list())


y = np.array(dataset[[label]]).squeeze()

if label == 'agitation' or label == 'agitation-next':
    y[y == -1] = 0
    y[y >= 1] = 1
elif label == 'agitation-four':
    y[y == -1] = 0
    y[y == -10] = 0

ids = np.array(dataset['id']).squeeze()
p = np.unique(ids, return_inverse=True)[1]

print(np.isnan(y).sum())
print(np.isnan(p).sum())

y = np.nan_to_num(y, nan=0)

print(np.isnan(y).sum())
print(np.isnan(p).sum())

print(np.unique(y))
print(y.sum())


# To Drop
dataset.drop(
    
    columns_count +
    

    # columns_contextual +

    # columns_statistical +
    
    # columns_physiology +


    columns_labels +
    columns_demographics + 
    columns_prediction

    , axis=1, inplace=True)

print(dataset.columns.to_list())

display('y: ', y)
display('p: ', p)
display('dataset: ', dataset)


['id', 'date', '6h', 'back-door', 'bathroom', 'bedroom', 'fridge-door', 'front-door', 'hallway', 'kitchen', 'lounge', 'total-events', 'unique-locations', 'active-location-ratio', 'private-to-public-ratio', 'location-entropy', 'location-dominance-ratio', 'back-and-forth-count', 'num-transitions', 'back-door-count-max', 'back-door-count-mean', 'back-door-count-std', 'back-door-count-sum', 'bathroom-count-max', 'bathroom-count-mean', 'bathroom-count-std', 'bathroom-count-sum', 'bedroom-count-max', 'bedroom-count-mean', 'bedroom-count-std', 'bedroom-count-sum', 'fridge-door-count-max', 'fridge-door-count-mean', 'fridge-door-count-std', 'fridge-door-count-sum', 'front-door-count-max', 'front-door-count-mean', 'front-door-count-std', 'front-door-count-sum', 'hallway-count-max', 'hallway-count-mean', 'hallway-count-std', 'hallway-count-sum', 'kitchen-count-max', 'kitchen-count-mean', 'kitchen-count-std', 'kitchen-count-sum', 'lounge-count-max', 'lounge-count-mean', 'lounge-count-std', 'lounge

'y: '

array([0, 0, 0, ..., 0, 0, 0], shape=(10790,))

'p: '

array([ 0,  0,  0, ..., 55, 55, 55], shape=(10790,))

'dataset: '

,id,date,6h,total-events,unique-locations,active-location-ratio,private-to-public-ratio,location-entropy,location-dominance-ratio,back-and-forth-count,...,lounge-count-std,lounge-count-sum,body-temperature,body-weight,diastolic-blood-pressure,heart-rate,muscle-mass,systolic-blood-pressure,total-body-water,skin-temperature
0,0697d,2019-06-28,00-06,0.0,0.0,0.000,1.0000,0.0000,0.0000,0.0,...,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0697d,2019-06-28,06-12,0.0,0.0,0.000,1.0000,0.0000,0.0000,0.0,...,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0697d,2019-06-28,12-18,221.0,8.0,1.000,0.1691,2.7340,0.2579,35.0,...,4.6999,49.0,36.1864,86.3,82.0,42.0,64.5,165.0,50.6,NaN
3,0697d,2019-06-28,18-24,101.0,7.0,0.875,0.0889,1.8916,0.4851,22.0,...,3.8130,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0697d,2019-06-29,00-06,3.0,1.0,0.125,1.0000,-0.0000,1.0000,0.0,...,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10785,fd100,2019-06-29,12-18,182.0,7.0,0.875,0.2283,2.5224,0.3077,31.0,...,4.4828,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10786,fd100,2019-06-29,18-24,114.0,7.0,0.875,0.3671,2.4755,0.2982,19.0,...,3.7583,34.0,36.2400,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10787,fd100,2019-06-30,00-06,0.0,0.0,0.000,1.0000,0.0000,0.0000,0.0,...,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10788,fd100,2019-06-30,06-12,146.0,7.0,0.875,0.5465,2.5379,0.2603,10.0,...,3.4683,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
# Imputation
import utils_data
importlib.reload(utils_data)
from utils_data import hierarchical_imputation



print(dataset.isna().sum().sum())

dataset = hierarchical_imputation(dataset)

print(dataset.isna().sum().sum())

dataset.drop(
    columns_indices,
    axis=1, inplace=True)

89869
0


In [73]:
x = np.array(dataset)

print(x.shape)
print(y.shape)

np.save(os.path.join('/home/ali/PycharmProjects/tihm/xyp', 'x.npy'), x)
np.save(os.path.join('/home/ali/PycharmProjects/tihm/xyp', 'y.npy'), y)
np.save(os.path.join('/home/ali/PycharmProjects/tihm/xyp', 'p.npy'), p)


print(np.unique(y))

feature_names = dataset.columns.values
print(feature_names)
display(dataset)

(10790, 48)
(10790,)
[0 1 2 3]
['total-events' 'unique-locations' 'active-location-ratio'
 'private-to-public-ratio' 'location-entropy' 'location-dominance-ratio'
 'back-and-forth-count' 'num-transitions' 'back-door-count-max'
 'back-door-count-mean' 'back-door-count-std' 'back-door-count-sum'
 'bathroom-count-max' 'bathroom-count-mean' 'bathroom-count-std'
 'bathroom-count-sum' 'bedroom-count-max' 'bedroom-count-mean'
 'bedroom-count-std' 'bedroom-count-sum' 'fridge-door-count-max'
 'fridge-door-count-mean' 'fridge-door-count-std' 'fridge-door-count-sum'
 'front-door-count-max' 'front-door-count-mean' 'front-door-count-std'
 'front-door-count-sum' 'hallway-count-max' 'hallway-count-mean'
 'hallway-count-std' 'hallway-count-sum' 'kitchen-count-max'
 'kitchen-count-mean' 'kitchen-count-std' 'kitchen-count-sum'
 'lounge-count-max' 'lounge-count-mean' 'lounge-count-std'
 'lounge-count-sum' 'body-temperature' 'body-weight'
 'diastolic-blood-pressure' 'heart-rate' 'muscle-mass'
 'systolic-b

,total-events,unique-locations,active-location-ratio,private-to-public-ratio,location-entropy,location-dominance-ratio,back-and-forth-count,num-transitions,back-door-count-max,back-door-count-mean,...,lounge-count-std,lounge-count-sum,body-temperature,body-weight,diastolic-blood-pressure,heart-rate,muscle-mass,systolic-blood-pressure,total-body-water,skin-temperature
0,0.0,0.0,0.000,1.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,...,0.0000,0.0,36.5675,69.1708,75.6146,72.1250,47.5853,136.4271,49.8206,34.2843
1,0.0,0.0,0.000,1.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,...,0.0000,0.0,36.5675,69.1708,75.6146,72.1250,47.5853,136.4271,49.8206,34.2843
2,221.0,8.0,1.000,0.1691,2.7340,0.2579,35.0,155.0,11.0,0.5455,...,4.6999,49.0,36.1864,86.3000,82.0000,42.0000,64.5000,165.0000,50.6000,34.2843
3,101.0,7.0,0.875,0.0889,1.8916,0.4851,22.0,61.0,1.0,0.0909,...,3.8130,31.0,36.5675,69.1708,75.6146,72.1250,47.5853,136.4271,49.8206,34.2843
4,3.0,1.0,0.125,1.0000,-0.0000,1.0000,0.0,1.0,0.0,0.0000,...,0.0000,0.0,36.5425,71.7159,74.1458,76.0469,48.6600,130.7188,51.7600,34.5600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10785,182.0,7.0,0.875,0.2283,2.5224,0.3077,31.0,139.0,0.0,0.0000,...,4.4828,46.0,36.5425,71.7159,74.1458,76.0469,48.6600,130.7188,51.7600,34.5600
10786,114.0,7.0,0.875,0.3671,2.4755,0.2982,19.0,92.0,0.0,0.0000,...,3.7583,34.0,36.2400,71.7159,74.1458,76.0469,48.6600,130.7188,51.7600,34.5600
10787,0.0,0.0,0.000,1.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,...,0.0000,0.0,36.7593,68.9053,73.7692,70.3526,46.2964,134.0000,51.1929,34.6478
10788,146.0,7.0,0.875,0.5465,2.5379,0.2603,10.0,110.0,0.0,0.0000,...,3.4683,31.0,36.7593,68.9053,73.7692,70.3526,46.2964,134.0000,51.1929,34.6478


In [74]:
# LightGBM

name = 'LightGBM'.lower()

Y_TRUES = np.empty([0])
Y_PROBS = []
Y_PREDS = np.empty([0])




# Explainability
# importances_list = []
# shap_list = []
# x_test_list = []


if kfold:
    cv = KFold(n_splits=5, shuffle=True, random_state=seed)
    split_iterator = cv.split(x)
else:
    cv = LeavePGroupsOut(n_groups=1)
    split_iterator = cv.split(x, y, groups=p)

for i, (train_idx, test_idx) in enumerate(split_iterator, start=1):
    participant = i if kfold else np.unique(p[test_idx])[0]



    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    print(participant, x_train.shape[0], x_test.shape[0])

    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    normalizer = MinMaxScaler()
    normalizer.fit(x_train)
    x_train = normalizer.transform(x_train)
    x_test = normalizer.transform(x_test)

    # Oversample class 1

    # ros = RandomOverSampler(sampling_strategy=.1, random_state=seed)
    # x_train, y_train = ros.fit_resample(x_train, y_train)

    # 1. Check original class distribution
    # print("Original class distribution:", Counter(y_train))
    # # 2. Define the minority class (adjust if needed)
    # minority_class = 1  # change this if your minority class label is different
    # current_minority_count = sum(y_train == minority_class)
    # # 3. Define desired new total count for the minority class (10x)
    # target_minority_count = current_minority_count * 10
    # # 4. Setup SMOTE with custom sampling strategy
    # smote = SMOTE(sampling_strategy={minority_class: target_minority_count}, random_state=seed)
    # # 5. Fit and resample
    # x_train, y_train = smote.fit_resample(x_train, y_train)
    # # 6. Confirm new distribution
    # print("Resampled class distribution:", Counter(y_train))



    # ----- LightGBM
    train_data = lgb.Dataset(x_train, label=y_train)
    test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
    params = {
        'verbose': -1,  # 👈 turn off training output
        'objective': 'multiclass',
        'num_class': 4,  # Specify number of classes
        'metric': 'multi_logloss',  # or 'auc' if you prefer
        'num_leaves': 64,
        'learning_rate': 0.01,
        'n_estimators': 100,
        
        'is_unbalance': weighted  # Automatically balances positive and negative classes

        # 'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum()
    }
    model = lgb.train(params, train_data, valid_sets=[train_data, test_data])
    y_probs = model.predict(x_test, num_iteration=model.best_iteration)


    Y_TRUES = np.append(Y_TRUES, y_test)
    Y_PROBS.append(y_probs)
    Y_PREDS = np.append(Y_PREDS, np.argmax(y_probs, axis=1))



    # Explainability
    # importances = model.feature_importance(importance_type='gain')
    # importances = model.feature_importance(importance_type='split')
    # importances_list.append(importances)


    # explainer = shap.TreeExplainer(model)
    # shap_values = explainer.shap_values(x_test)
    # shap_list.append(shap_values)
    # x_test_list.append(x_test)





Y_PROBS = np.concatenate(Y_PROBS, axis=0)


indx = Y_TRUES.argsort()
Y_TRUES = Y_TRUES[indx]
Y_PROBS = Y_PROBS[indx]
Y_PREDS = Y_PREDS[indx]


# Evaluation metrics (multiclass)
auc_roc_macro = roc_auc_score(Y_TRUES, Y_PROBS, multi_class='ovr', average='macro')
auc_roc_weighted = roc_auc_score(Y_TRUES, Y_PROBS, multi_class='ovr', average='weighted')
auc_pr_macro = average_precision_score(Y_TRUES, Y_PROBS, average='macro')
auc_pr_weighted = average_precision_score(Y_TRUES, Y_PROBS, average='weighted')
acc = accuracy_score(Y_TRUES, Y_PREDS)
# Confusion matrix
cm = confusion_matrix(Y_TRUES, Y_PREDS)
print(cm)
cm_str = '\n'.join(['[' + ' '.join(map(str, row)) + ']' for row in cm])

cm_flat = cm.flatten()

results = {
    'auc-roc-macro': f"{auc_roc_macro:.4f}",
    'auc-roc-weighted': f"{auc_roc_weighted:.4f}",
    'auc-pr-macro': f"{auc_pr_macro:.4f}",
    'auc-pr-weighted': f"{auc_pr_weighted:.4f}",
    'accuracy': f"{acc:.4f}",
    'confusion_matrix': cm_str
}

for i, val in enumerate(cm_flat):
    results[f'cm-{i}'] = str(int(val))

results = pd.DataFrame([results])
results.to_csv(os.path.join(results_dir, name + '.csv'), index=False)
display(results)


0 10779 11
1 10617 173
2 10488 302
3 10599 191
4 10604 186
5 10767 23
6 10437 353
7 10541 249
8 10500 290
9 10611 179
10 10771 19
11 10560 230
12 10532 258
13 10716 74
14 10524 266
15 10548 242
16 10665 125
17 10508 282
18 10668 122
19 10570 220
20 10588 202
21 10521 269
22 10627 163
23 10600 190
24 10771 19
25 10520 270
26 10495 295
27 10520 270
28 10605 185
29 10616 174
30 10576 214
31 10556 234
32 10687 103
33 10560 230
34 10625 165
35 10751 39
36 10525 265
37 10436 354
38 10621 169
39 10463 327
40 10524 266
41 10584 206
42 10632 158
43 10520 270
44 10728 62
45 10699 91
46 10763 27
47 10580 210
48 10672 118
49 10528 262
50 10627 163
51 10541 249
52 10502 288
53 10575 215
54 10532 258
55 10775 15
[[10563     3     0     0]
 [  117     0     0     0]
 [   89     0     0     0]
 [   18     0     0     0]]


,auc-roc-macro,auc-roc-weighted,auc-pr-macro,auc-pr-weighted,accuracy,confusion_matrix,cm-0,cm-1,cm-2,cm-3,...,cm-6,cm-7,cm-8,cm-9,cm-10,cm-11,cm-12,cm-13,cm-14,cm-15
0,0.8657,0.8434,0.3027,0.9719,0.9790,[10563 3 0 0]\n[117 0 0 0]\n[89 0 0 0]\n[18 0 ...,10563,3,0,0,...,0,0,89,0,0,0,18,0,0,0


In [75]:
# # Explainability

# print(len(feature_names), np.vstack(importances_list).mean(axis=0).round(4).shape, x.shape, y.shape, x_test.shape, y_test.shape)

# importance_df = pd.DataFrame({
#     'feature': feature_names,
#     'importance': np.vstack(importances_list).mean(axis=0).round(4)
# }).sort_values(by='importance', ascending=False)
# display(importance_df)
# importance_df.to_csv('importance.csv', index=False)


# # Create DataFrame for summary
# shap_df = pd.DataFrame({
#     'feature': feature_names,
#     'mean_abs_shap': np.abs(np.vstack(shap_list)).mean(axis=0).round(4)
# }).sort_values(by='mean_abs_shap', ascending=False)
# shap_df.to_csv('importance-shap.csv', index=False)

# print(np.vstack(x_test_list).shape)

# shap.summary_plot(shap_values, pd.DataFrame(x_test, columns=feature_names), max_display=16)
# shap.summary_plot(np.vstack(shap_list), pd.DataFrame(np.vstack(x_test_list), columns=feature_names), max_display=16)



In [76]:
# Transformer

if not weighted:

    name = 'Transformer'.lower()

    Y_TRUES = np.empty([0])
    Y_PROBS = []
    Y_PREDS = np.empty([0])


    if kfold:
        cv = KFold(n_splits=5, shuffle=True, random_state=seed)
        split_iterator = cv.split(x)
    else:
        cv = LeavePGroupsOut(n_groups=1)
        split_iterator = cv.split(x, y, groups=p)

    for i, (train_idx, test_idx) in enumerate(split_iterator, start=1):
        participant = i if kfold else np.unique(p[test_idx])[0]

        x_train, x_test = x[train_idx], x[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        print(participant, x_train.shape[0], x_test.shape[0])

        scaler = StandardScaler()
        scaler.fit(x_train)
        x_train = scaler.transform(x_train)
        x_test = scaler.transform(x_test)

        normalizer = MinMaxScaler()
        normalizer.fit(x_train)
        x_train = normalizer.transform(x_train)
        x_test = normalizer.transform(x_test)

        # Oversample class 1

        # ros = RandomOverSampler(sampling_strategy=.1, random_state=seed)
        # x_train, y_train = ros.fit_resample(x_train, y_train)

        # # 1. Check original class distribution
        # print("Original class distribution:", Counter(y_train))
        # # 2. Define the minority class (adjust if needed)
        # minority_class = 1  # change this if your minority class label is different
        # current_minority_count = sum(y_train == minority_class)
        # # 3. Define desired new total count for the minority class (10x)
        # target_minority_count = current_minority_count * 10
        # # 4. Setup SMOTE with custom sampling strategy
        # smote = SMOTE(sampling_strategy={minority_class: target_minority_count}, random_state=seed)
        # # 5. Fit and resample
        # x_train, y_train = smote.fit_resample(x_train, y_train)
        # # 6. Confirm new distribution
        # print("Resampled class distribution:", Counter(y_train))



        # ----- LightGBM
        # train_data = lgb.Dataset(x_train, label=y_train)
        # test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
        # params = {
        #     'verbose': -1,  # 👈 turn off training output
        #     'objective': 'binary',
        #     'metric': 'binary_logloss',  # or 'auc' if you prefer
        #     'num_leaves': 64,
        #     'learning_rate': 0.01,
        #     'n_estimators': 100,
        #     # 'is_unbalance': True  # Automatically balances positive and negative classes
        #     # 'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum()
        # }
        # bst = lgb.train(params, train_data, valid_sets=[train_data, test_data])
        # y_probs = bst.predict(x_test, num_iteration=bst.best_iteration)


        # ----- Transformer
        model = TabPFNClassifier(
            device='cuda',
            random_state=seed,
            n_estimators = 4,
            ignore_pretraining_limits=True)
        model.fit(x_train, y_train)
        y_probs = model.predict_proba(x_test)


        Y_TRUES = np.append(Y_TRUES, y_test)
        Y_PROBS.append(y_probs)
        Y_PREDS = np.append(Y_PREDS, np.argmax(y_probs, axis=1))

    Y_PROBS = np.concatenate(Y_PROBS, axis=0)


    indx = Y_TRUES.argsort()
    Y_TRUES = Y_TRUES[indx]
    Y_PROBS = Y_PROBS[indx]
    Y_PREDS = Y_PREDS[indx]


# Evaluation metrics (multiclass)
auc_roc_macro = roc_auc_score(Y_TRUES, Y_PROBS, multi_class='ovr', average='macro')
auc_roc_weighted = roc_auc_score(Y_TRUES, Y_PROBS, multi_class='ovr', average='weighted')
auc_pr_macro = average_precision_score(Y_TRUES, Y_PROBS, average='macro')
auc_pr_weighted = average_precision_score(Y_TRUES, Y_PROBS, average='weighted')
acc = accuracy_score(Y_TRUES, Y_PREDS)
# Confusion matrix
cm = confusion_matrix(Y_TRUES, Y_PREDS)
print(cm)
cm_str = '\n'.join(['[' + ' '.join(map(str, row)) + ']' for row in cm])

cm_flat = cm.flatten()

results = {
    'auc-roc-macro': f"{auc_roc_macro:.4f}",
    'auc-roc-weighted': f"{auc_roc_weighted:.4f}",
    'auc-pr-macro': f"{auc_pr_macro:.4f}",
    'auc-pr-weighted': f"{auc_pr_weighted:.4f}",
    'accuracy': f"{acc:.4f}",
    'confusion_matrix': cm_str
}

for i, val in enumerate(cm_flat):
    results[f'cm-{i}'] = str(int(val))

results = pd.DataFrame([results])
results.to_csv(os.path.join(results_dir, name + '.csv'), index=False)
display(results)

[[10563     3     0     0]
 [  117     0     0     0]
 [   89     0     0     0]
 [   18     0     0     0]]


,auc-roc-macro,auc-roc-weighted,auc-pr-macro,auc-pr-weighted,accuracy,confusion_matrix,cm-0,cm-1,cm-2,cm-3,...,cm-6,cm-7,cm-8,cm-9,cm-10,cm-11,cm-12,cm-13,cm-14,cm-15
0,0.8657,0.8434,0.3027,0.9719,0.9790,[10563 3 0 0]\n[117 0 0 0]\n[89 0 0 0]\n[18 0 ...,10563,3,0,0,...,0,0,89,0,0,0,18,0,0,0


In [77]:
# Gradient Boosting Classifier

name = 'Gradient-Boosting'.lower()

Y_TRUES = np.empty([0])
Y_PROBS = []
Y_PREDS = np.empty([0])


if kfold:
    cv = KFold(n_splits=5, shuffle=True, random_state=seed)
    split_iterator = cv.split(x)
else:
    cv = LeavePGroupsOut(n_groups=1)
    split_iterator = cv.split(x, y, groups=p)

for i, (train_idx, test_idx) in enumerate(split_iterator, start=1):
    participant = i if kfold else np.unique(p[test_idx])[0]

    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    print(participant, x_train.shape[0], x_test.shape[0])

    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    normalizer = MinMaxScaler()
    normalizer.fit(x_train)
    x_train = normalizer.transform(x_train)
    x_test = normalizer.transform(x_test)

    # Oversample class 1

    # ros = RandomOverSampler(sampling_strategy=.1, random_state=seed)
    # x_train, y_train = ros.fit_resample(x_train, y_train)

    # # 1. Check original class distribution
    # print("Original class distribution:", Counter(y_train))
    # # 2. Define the minority class (adjust if needed)
    # minority_class = 1  # change this if your minority class label is different
    # current_minority_count = sum(y_train == minority_class)
    # # 3. Define desired new total count for the minority class (10x)
    # target_minority_count = current_minority_count * 10
    # # 4. Setup SMOTE with custom sampling strategy
    # smote = SMOTE(sampling_strategy={minority_class: target_minority_count}, random_state=seed)
    # # 5. Fit and resample
    # x_train, y_train = smote.fit_resample(x_train, y_train)
    # # 6. Confirm new distribution
    # print("Resampled class distribution:", Counter(y_train))


    # ----- LightGBM
    # train_data = lgb.Dataset(x_train, label=y_train)
    # test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
    # params = {
    #     'verbose': -1,  # 👈 turn off training output
    #     'objective': 'binary',
    #     'metric': 'binary_logloss',  # or 'auc' if you prefer
    #     'num_leaves': 64,
    #     'learning_rate': 0.01,
    #     'n_estimators': 100,
    #     # 'is_unbalance': True  # Automatically balances positive and negative classes
    #     # 'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum()
    # }
    # bst = lgb.train(params, train_data, valid_sets=[train_data, test_data])
    # y_probs = bst.predict(x_test, num_iteration=bst.best_iteration)


    # ----- Transformer
    # model = TabPFNClassifier(
    #     device='cuda',
    #     random_state=seed,
    #     n_estimators = 4,
    #     ignore_pretraining_limits=True)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Gradient Boosting Classifier
    model = GradientBoostingClassifier(
        n_estimators=100,  # match LightGBM
        learning_rate=0.001,  # match LightGBM
        max_depth=4,  # similar to LightGBM default tree depth
        subsample=1.0,  # default
        random_state=seed
    )
    if weighted:
        model.fit(x_train, y_train, sample_weight=compute_sample_weight(class_weight='balanced', y=y_train))
    else:
        model.fit(x_train, y_train)
    
    y_probs = model.predict_proba(x_test)
    y_preds = model.predict(x_test)
    

    Y_TRUES = np.append(Y_TRUES, y_test)
    Y_PROBS.append(y_probs)
    Y_PREDS = np.append(Y_PREDS, y_preds)

Y_PROBS = np.concatenate(Y_PROBS, axis=0)


indx = Y_TRUES.argsort()
Y_TRUES = Y_TRUES[indx]
Y_PROBS = Y_PROBS[indx]
Y_PREDS = Y_PREDS[indx]


# Evaluation metrics (multiclass)
auc_roc_macro = roc_auc_score(Y_TRUES, Y_PROBS, multi_class='ovr', average='macro')
auc_roc_weighted = roc_auc_score(Y_TRUES, Y_PROBS, multi_class='ovr', average='weighted')
auc_pr_macro = average_precision_score(Y_TRUES, Y_PROBS, average='macro')
auc_pr_weighted = average_precision_score(Y_TRUES, Y_PROBS, average='weighted')
acc = accuracy_score(Y_TRUES, Y_PREDS)
# Confusion matrix
cm = confusion_matrix(Y_TRUES, Y_PREDS)
print(cm)
cm_str = '\n'.join(['[' + ' '.join(map(str, row)) + ']' for row in cm])

cm_flat = cm.flatten()

results = {
    'auc-roc-macro': f"{auc_roc_macro:.4f}",
    'auc-roc-weighted': f"{auc_roc_weighted:.4f}",
    'auc-pr-macro': f"{auc_pr_macro:.4f}",
    'auc-pr-weighted': f"{auc_pr_weighted:.4f}",
    'accuracy': f"{acc:.4f}",
    'confusion_matrix': cm_str
}

for i, val in enumerate(cm_flat):
    results[f'cm-{i}'] = str(int(val))

results = pd.DataFrame([results])
results.to_csv(os.path.join(results_dir, name + '.csv'), index=False)
display(results)

0 10779 11
1 10617 173
2 10488 302
3 10599 191
4 10604 186
5 10767 23
6 10437 353
7 10541 249
8 10500 290
9 10611 179
10 10771 19
11 10560 230
12 10532 258
13 10716 74
14 10524 266
15 10548 242
16 10665 125
17 10508 282
18 10668 122
19 10570 220
20 10588 202
21 10521 269
22 10627 163
23 10600 190
24 10771 19
25 10520 270
26 10495 295
27 10520 270
28 10605 185
29 10616 174
30 10576 214
31 10556 234
32 10687 103
33 10560 230
34 10625 165
35 10751 39
36 10525 265
37 10436 354
38 10621 169
39 10463 327
40 10524 266
41 10584 206
42 10632 158
43 10520 270
44 10728 62
45 10699 91
46 10763 27
47 10580 210
48 10672 118
49 10528 262
50 10627 163
51 10541 249
52 10502 288
53 10575 215
54 10532 258
55 10775 15
[[8793  607 1023  143]
 [  24   66   13   14]
 [  31   20   29    9]
 [   3    8    3    4]]


,auc-roc-macro,auc-roc-weighted,auc-pr-macro,auc-pr-weighted,accuracy,confusion_matrix,cm-0,cm-1,cm-2,cm-3,...,cm-6,cm-7,cm-8,cm-9,cm-10,cm-11,cm-12,cm-13,cm-14,cm-15
0,0.7645,0.8288,0.2809,0.9735,0.8241,[8793 607 1023 143]\n[24 66 13 14]\n[31 20 29 ...,8793,607,1023,143,...,13,14,31,20,29,9,3,8,3,4


In [78]:
# Logistic Regression

name = 'Logistic-Regression'.lower()

Y_TRUES = np.empty([0])
Y_PROBS = []
Y_PREDS = np.empty([0])


if kfold:
    cv = KFold(n_splits=5, shuffle=True, random_state=seed)
    split_iterator = cv.split(x)
else:
    cv = LeavePGroupsOut(n_groups=1)
    split_iterator = cv.split(x, y, groups=p)

for i, (train_idx, test_idx) in enumerate(split_iterator, start=1):
    participant = i if kfold else np.unique(p[test_idx])[0]

    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    print(participant, x_train.shape[0], x_test.shape[0])

    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    normalizer = MinMaxScaler()
    normalizer.fit(x_train)
    x_train = normalizer.transform(x_train)
    x_test = normalizer.transform(x_test)

    # Oversample class 1

    # ros = RandomOverSampler(sampling_strategy=.1, random_state=seed)
    # x_train, y_train = ros.fit_resample(x_train, y_train)

    # 1. Check original class distribution
    # print("Original class distribution:", Counter(y_train))
    # # 2. Define the minority class (adjust if needed)
    # minority_class = 1  # change this if your minority class label is different
    # current_minority_count = sum(y_train == minority_class)
    # # 3. Define desired new total count for the minority class (10x)
    # target_minority_count = current_minority_count * 10
    # # 4. Setup SMOTE with custom sampling strategy
    # smote = SMOTE(sampling_strategy={minority_class: target_minority_count}, random_state=seed)
    # # 5. Fit and resample
    # x_train, y_train = smote.fit_resample(x_train, y_train)
    # # 6. Confirm new distribution
    # print("Resampled class distribution:", Counter(y_train))


    # Undersample class 0
    # reduction_percent = 99
    # print("Original class distribution:", Counter(y_train))    
    # class_0_idx = np.where(y_train == 0)[0]
    # class_1_idx = np.where(y_train == 1)[0]
    # # How many class 0 samples to keep
    # n_keep = int(len(class_0_idx) * (1 - reduction_percent / 100.0))
    # sampled_0_idx = np.random.choice(class_0_idx, size=n_keep, replace=False)
    # # Combine sampled class 0 with all class 1
    # final_idx = np.concatenate([sampled_0_idx, class_1_idx])
    # x_train = x_train[final_idx]
    # y_train =  y_train[final_idx]
    # print("Resampled class distribution:", Counter(y_train))


    # ----- LightGBM
    # train_data = lgb.Dataset(x_train, label=y_train)
    # test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
    # params = {
    #     'verbose': -1,  # 👈 turn off training output
    #     'objective': 'binary',
    #     'metric': 'binary_logloss',  # or 'auc' if you prefer
    #     'num_leaves': 64,
    #     'learning_rate': 0.01,
    #     'n_estimators': 100,
    #     # 'is_unbalance': True  # Automatically balances positive and negative classes
    #     # 'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum()
    # }
    # bst = lgb.train(params, train_data, valid_sets=[train_data, test_data])
    # y_probs = bst.predict(x_test, num_iteration=bst.best_iteration)


    # ----- Transformer
    # model = TabPFNClassifier(
    #     device='cuda',
    #     random_state=seed,
    #     n_estimators = 4,
    #     ignore_pretraining_limits=True)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Gradient Boosting Classifier
    # model = GradientBoostingClassifier(
    #     n_estimators=100,  # match LightGBM
    #     learning_rate=0.001,  # match LightGBM
    #     max_depth=4,  # similar to LightGBM default tree depth
    #     subsample=1.0,  # default
    #     random_state=seed
    # )
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]
    

    # ----- XGBoost
    # model = XGBClassifier()
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Random Forest
    # model = RandomForestClassifier(
    #     n_estimators=250,  # number of trees
    #     max_depth=None,  # let the trees grow fully
    #     random_state=seed
    # )
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- SVM
    # model = SVC(kernel='rbf', probability=True, random_state=seed)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- DT
    # model = DecisionTreeClassifier(criterion="log_loss", max_depth=512, random_state=seed)
    # model.fit(x_train, y_train)
    # y_probs = model.predict(x_test)


    # MLP
    # model = MLPClassifier(
    #     hidden_layer_sizes=(128, 64),    # 2 hidden layers: 64 and 32 neurons
    #     activation='relu',              # good default: 'relu'
    #     solver='adam',                  # optimizer
    #     alpha=0.0001,                   # L2 regularization
    #     learning_rate_init=0.001,
    #     max_iter=500,
    #     early_stopping=True,
    #     random_state=seed,
    #     verbose=True,
    # )
    # model.fit(x_train, y_train)    
    # y_probs = model.predict_proba(x_test)[:, 1]
    # y_preds = model.predict(x_test)


    # LR
    model = LogisticRegression(
    penalty='l2',               # regularization (default)
    C=1.0,                      # inverse of regularization strength
    solver='lbfgs',             # optimizer (good default for small/medium data)
    max_iter=1000,

    class_weight='balanced' if weighted else None,

    random_state=seed)
    model.fit(x_train, y_train)
    y_probs = model.predict_proba(x_test)
    y_preds = model.predict(x_test)


    Y_TRUES = np.append(Y_TRUES, y_test)
    Y_PROBS.append(y_probs)
    Y_PREDS = np.append(Y_PREDS, y_preds)

Y_PROBS = np.concatenate(Y_PROBS, axis=0)


indx = Y_TRUES.argsort()
Y_TRUES = Y_TRUES[indx]
Y_PROBS = Y_PROBS[indx]
Y_PREDS = Y_PREDS[indx]


# Evaluation metrics (multiclass)
auc_roc_macro = roc_auc_score(Y_TRUES, Y_PROBS, multi_class='ovr', average='macro')
auc_roc_weighted = roc_auc_score(Y_TRUES, Y_PROBS, multi_class='ovr', average='weighted')
auc_pr_macro = average_precision_score(Y_TRUES, Y_PROBS, average='macro')
auc_pr_weighted = average_precision_score(Y_TRUES, Y_PROBS, average='weighted')
acc = accuracy_score(Y_TRUES, Y_PREDS)
# Confusion matrix
cm = confusion_matrix(Y_TRUES, Y_PREDS)
print(cm)
cm_str = '\n'.join(['[' + ' '.join(map(str, row)) + ']' for row in cm])

cm_flat = cm.flatten()

results = {
    'auc-roc-macro': f"{auc_roc_macro:.4f}",
    'auc-roc-weighted': f"{auc_roc_weighted:.4f}",
    'auc-pr-macro': f"{auc_pr_macro:.4f}",
    'auc-pr-weighted': f"{auc_pr_weighted:.4f}",
    'accuracy': f"{acc:.4f}",
    'confusion_matrix': cm_str
}

for i, val in enumerate(cm_flat):
    results[f'cm-{i}'] = str(int(val))

results = pd.DataFrame([results])
results.to_csv(os.path.join(results_dir, name + '.csv'), index=False)
display(results)

0 10779 11
1 10617 173
2 10488 302
3 10599 191
4 10604 186
5 10767 23
6 10437 353
7 10541 249
8 10500 290
9 10611 179
10 10771 19
11 10560 230
12 10532 258
13 10716 74
14 10524 266
15 10548 242
16 10665 125
17 10508 282
18 10668 122
19 10570 220
20 10588 202
21 10521 269
22 10627 163
23 10600 190
24 10771 19
25 10520 270
26 10495 295
27 10520 270
28 10605 185
29 10616 174
30 10576 214
31 10556 234
32 10687 103
33 10560 230
34 10625 165
35 10751 39
36 10525 265
37 10436 354
38 10621 169
39 10463 327
40 10524 266
41 10584 206
42 10632 158
43 10520 270
44 10728 62
45 10699 91
46 10763 27
47 10580 210
48 10672 118
49 10528 262
50 10627 163
51 10541 249
52 10502 288
53 10575 215
54 10532 258
55 10775 15
[[8156  743 1453  214]
 [  14   66   23   14]
 [  16   27   34   12]
 [   0   10    2    6]]


,auc-roc-macro,auc-roc-weighted,auc-pr-macro,auc-pr-weighted,accuracy,confusion_matrix,cm-0,cm-1,cm-2,cm-3,...,cm-6,cm-7,cm-8,cm-9,cm-10,cm-11,cm-12,cm-13,cm-14,cm-15
0,0.8354,0.8858,0.2991,0.9777,0.7657,[8156 743 1453 214]\n[14 66 23 14]\n[16 27 34 ...,8156,743,1453,214,...,23,14,16,27,34,12,0,10,2,6


In [79]:
# Naive Bayes

name = 'Naive-Bayes'.lower()

Y_TRUES = np.empty([0])
Y_PROBS = []
Y_PREDS = np.empty([0])


if kfold:
    cv = KFold(n_splits=5, shuffle=True, random_state=seed)
    split_iterator = cv.split(x)
else:
    cv = LeavePGroupsOut(n_groups=1)
    split_iterator = cv.split(x, y, groups=p)

for i, (train_idx, test_idx) in enumerate(split_iterator, start=1):
    participant = i if kfold else np.unique(p[test_idx])[0]

    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    print(participant, x_train.shape[0], x_test.shape[0])

    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    normalizer = MinMaxScaler()
    normalizer.fit(x_train)
    x_train = normalizer.transform(x_train)
    x_test = normalizer.transform(x_test)

    # Oversample class 1

    # ros = RandomOverSampler(sampling_strategy=.1, random_state=seed)
    # x_train, y_train = ros.fit_resample(x_train, y_train)

    # # 1. Check original class distribution
    # print("Original class distribution:", Counter(y_train))
    # # 2. Define the minority class (adjust if needed)
    # minority_class = 1  # change this if your minority class label is different
    # current_minority_count = sum(y_train == minority_class)
    # # 3. Define desired new total count for the minority class (10x)
    # target_minority_count = current_minority_count * 10
    # # 4. Setup SMOTE with custom sampling strategy
    # smote = SMOTE(sampling_strategy={minority_class: target_minority_count}, random_state=seed)
    # # 5. Fit and resample
    # x_train, y_train = smote.fit_resample(x_train, y_train)
    # # 6. Confirm new distribution
    # print("Resampled class distribution:", Counter(y_train))


    # Undersample class 0
    # data_percent = 1
    # reduction_percent = 100 - data_percent
    # print("Original class distribution:", Counter(y_train))    
    # class_0_idx = np.where(y_train == 0)[0]
    # class_1_idx = np.where(y_train == 1)[0]
    # # How many class 0 samples to keep
    # n_keep = int(len(class_0_idx) * (1 - reduction_percent / 100.0))
    # sampled_0_idx = np.random.choice(class_0_idx, size=n_keep, replace=False)
    # # Combine sampled class 0 with all class 1
    # final_idx = np.concatenate([sampled_0_idx, class_1_idx])
    # x_train = x_train[final_idx]
    # y_train =  y_train[final_idx]
    # print("Resampled class distribution:", Counter(y_train))


    # ----- LightGBM
    # train_data = lgb.Dataset(x_train, label=y_train)
    # test_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
    # params = {
    #     'verbose': -1,  # 👈 turn off training output
    #     'objective': 'binary',
    #     'metric': 'binary_logloss',  # or 'auc' if you prefer
    #     'num_leaves': 64,
    #     'learning_rate': 0.01,
    #     'n_estimators': 100,
    #     # 'is_unbalance': True  # Automatically balances positive and negative classes
    #     # 'scale_pos_weight': (y_train == 0).sum() / (y_train == 1).sum()
    # }
    # bst = lgb.train(params, train_data, valid_sets=[train_data, test_data])
    # y_probs = bst.predict(x_test, num_iteration=bst.best_iteration)


    # ----- Transformer
    # model = TabPFNClassifier(
    #     device='cuda',
    #     random_state=seed,
    #     n_estimators = 4,
    #     ignore_pretraining_limits=True)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Gradient Boosting Classifier
    # model = GradientBoostingClassifier(
    #     n_estimators=100,  # match LightGBM
    #     learning_rate=0.001,  # match LightGBM
    #     max_depth=4,  # similar to LightGBM default tree depth
    #     subsample=1.0,  # default
    #     random_state=seed
    # )
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]
    

    # ----- XGBoost
    # model = XGBClassifier()
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- Random Forest
    # model = RandomForestClassifier(
    #     n_estimators=250,  # number of trees
    #     max_depth=None,  # let the trees grow fully
    #     random_state=seed
    # )
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- SVM
    # model = SVC(kernel='rbf', probability=True, random_state=seed)
    # model.fit(x_train, y_train)
    # y_probs = model.predict_proba(x_test)[:, 1]


    # ----- DT
    # model = DecisionTreeClassifier(criterion="log_loss", max_depth=512, random_state=seed)
    # model.fit(x_train, y_train)
    # y_probs = model.predict(x_test)


    # MLP
    # model = MLPClassifier(
    #     hidden_layer_sizes=(128, 64),    # 2 hidden layers: 64 and 32 neurons
    #     activation='relu',              # good default: 'relu'
    #     solver='adam',                  # optimizer
    #     alpha=0.0001,                   # L2 regularization
    #     learning_rate_init=0.001,
    #     max_iter=500,
    #     early_stopping=True,
    #     random_state=seed,
    #     verbose=True,
    # )
    # model.fit(x_train, y_train)    
    # y_probs = model.predict_proba(x_test)[:, 1]
    # y_preds = model.predict(x_test)


    # LR
    # model = LogisticRegression(
    # penalty='l2',               # regularization (default)
    # C=1.0,                      # inverse of regularization strength
    # solver='lbfgs',             # optimizer (good default for small/medium data)
    # max_iter=1000,
    # class_weight='balanced',
    # random_state=seed)
    # model.fit(x_train, y_train)    
    # y_probs = model.predict_proba(x_test)[:, 1]
    # y_preds = model.predict(x_test)


    # NB
    model = GaussianNB()
    model.fit(x_train,
              y_train,

              sample_weight=compute_sample_weight(class_weight='balanced', y=y_train) if weighted else None,

              )
    y_probs = model.predict_proba(x_test)
    y_preds = model.predict(x_test)


    Y_TRUES = np.append(Y_TRUES, y_test)
    Y_PROBS.append(y_probs)
    Y_PREDS = np.append(Y_PREDS, y_preds)

Y_PROBS = np.concatenate(Y_PROBS, axis=0)


indx = Y_TRUES.argsort()
Y_TRUES = Y_TRUES[indx]
Y_PROBS = Y_PROBS[indx]
Y_PREDS = Y_PREDS[indx]


# Evaluation metrics (multiclass)
auc_roc_macro = roc_auc_score(Y_TRUES, Y_PROBS, multi_class='ovr', average='macro')
auc_roc_weighted = roc_auc_score(Y_TRUES, Y_PROBS, multi_class='ovr', average='weighted')
auc_pr_macro = average_precision_score(Y_TRUES, Y_PROBS, average='macro')
auc_pr_weighted = average_precision_score(Y_TRUES, Y_PROBS, average='weighted')
acc = accuracy_score(Y_TRUES, Y_PREDS)
# Confusion matrix
cm = confusion_matrix(Y_TRUES, Y_PREDS)
print(cm)
cm_str = '\n'.join(['[' + ' '.join(map(str, row)) + ']' for row in cm])

cm_flat = cm.flatten()

results = {
    'auc-roc-macro': f"{auc_roc_macro:.4f}",
    'auc-roc-weighted': f"{auc_roc_weighted:.4f}",
    'auc-pr-macro': f"{auc_pr_macro:.4f}",
    'auc-pr-weighted': f"{auc_pr_weighted:.4f}",
    'accuracy': f"{acc:.4f}",
    'confusion_matrix': cm_str
}

for i, val in enumerate(cm_flat):
    results[f'cm-{i}'] = str(int(val))

results = pd.DataFrame([results])
results.to_csv(os.path.join(results_dir, name + '.csv'), index=False)
display(results)

0 10779 11
1 10617 173
2 10488 302
3 10599 191
4 10604 186
5 10767 23
6 10437 353
7 10541 249
8 10500 290
9 10611 179
10 10771 19
11 10560 230
12 10532 258
13 10716 74
14 10524 266
15 10548 242
16 10665 125
17 10508 282
18 10668 122
19 10570 220
20 10588 202
21 10521 269
22 10627 163
23 10600 190
24 10771 19
25 10520 270
26 10495 295
27 10520 270
28 10605 185
29 10616 174
30 10576 214
31 10556 234
32 10687 103
33 10560 230
34 10625 165
35 10751 39
36 10525 265
37 10436 354
38 10621 169
39 10463 327
40 10524 266
41 10584 206
42 10632 158
43 10520 270
44 10728 62
45 10699 91
46 10763 27
47 10580 210
48 10672 118
49 10528 262
50 10627 163
51 10541 249
52 10502 288
53 10575 215
54 10532 258
55 10775 15
[[7178 1329 1805  254]
 [  13   69   15   20]
 [  12   23   37   17]
 [   0    9    3    6]]


,auc-roc-macro,auc-roc-weighted,auc-pr-macro,auc-pr-weighted,accuracy,confusion_matrix,cm-0,cm-1,cm-2,cm-3,...,cm-6,cm-7,cm-8,cm-9,cm-10,cm-11,cm-12,cm-13,cm-14,cm-15
0,0.8158,0.8632,0.2720,0.9761,0.6756,[7178 1329 1805 254]\n[13 69 15 20]\n[12 23 37...,7178,1329,1805,254,...,15,20,12,23,37,17,0,9,3,6


In [80]:
# Combine
import utils_data
importlib.reload(utils_data)
from utils_data import combine_single_row_csvs


combine_single_row_csvs(results_dir, os.path.join(results_dir,os.path.basename(results_dir)))